# Chatbot

In [1]:
import pandas as pd
import re
from rapidfuzz import process

In [2]:
file_path = "financial_data_processed.csv"
data = pd.read_csv(file_path)
data["Company"] = data["Company"].str.lower()
data["Year"] = data["Year"].astype(int)


## Define supported companies and metrics and helper functions (Preprocess user input, Extract company, metric, and year, Fuzzy matching)

In [3]:
companies = ["apple", "microsoft", "tesla"]
metrics = [
    "revenue", "revenue growth", "net income", "net income growth",
    "profitability", "debt-to-asset ratio", "cash flow efficiency",
    "financial health index", "market revenue share", "profitable", "revenue cagr"
]

# Preprocessing and helper functions
def preprocess_query(user_query):
    cleaned_query = user_query.lower()
    cleaned_query = re.sub(r"\b(of|in|for|the|is|what|are|and|between)\b", "", cleaned_query)
    return cleaned_query.strip()

def fuzzy_match(user_input, choices, threshold=60):
    # Sort choices by length to prioritize longer phrases
    sorted_choices = sorted(choices, key=len, reverse=True)
    match, score, _ = process.extractOne(user_input, sorted_choices)
    print(f"Matching '{user_input}' against choices: {sorted_choices}. Best match: {match}, Score: {score}")
    
    # Ensure the match score is above the threshold
    return match if score >= threshold else None


def extract_information(user_query):
    # Extract all years mentioned in the query
    years = re.findall(r"\b\d{4}\b", user_query)

    # Extract all mentioned companies by checking for matches in the query
    companies_in_query = [company for company in companies if company in user_query]

    # Extract metrics by matching keywords in the query
    metrics_in_query = [metric for metric in metrics if metric in user_query]

    print(f"Processed Query: {user_query}")
    print(f"Extracted Companies: {companies_in_query}, Metrics: {metrics_in_query}, Years: {years}")
    
    return companies_in_query, metrics_in_query, years


## Query Functions

In [4]:
# Query Functions

def total_revenue_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        revenue = result["Total Revenue (in millions)"].values[0]
        return f"The total revenue for {company.title()} in {year} is ${revenue} million."
    else:
        return f"No revenue data available for {company.title()} in {year}."

def revenue_growth_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        growth = result["Revenue Growth (%)"].values[0]
        return f"The revenue growth for {company.title()} in {year} is {growth:.2f}%."
    else:
        return f"No revenue growth data available for {company.title()} in {year}."

def net_income_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        net_income = result["Net Income (in millions)"].values[0]
        return f"The net income for {company.title()} in {year} is ${net_income} million."
    else:
        return f"No net income data available for {company.title()} in {year}."

def net_income_growth_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        growth = result["Net Income Growth (%)"].values[0]
        return f"The net income growth for {company.title()} in {year} is {growth:.2f}%."
    else:
        return f"No net income growth data available for {company.title()} in {year}."

def profitability_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        profitability = result["Profitability (%)"].values[0]
        return f"The profitability for {company.title()} in {year} is {profitability:.2f}%."
    else:
        return f"No profitability data available for {company.title()} in {year}."

def debt_to_asset_ratio_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        ratio = result["Debt-to-Asset Ratio"].values[0]
        return f"The debt-to-asset ratio for {company.title()} in {year} is {ratio:.2f}."
    else:
        return f"No debt-to-asset ratio data available for {company.title()} in {year}."

def financial_health_index_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        fhi = result["Financial Health Index"].values[0]
        return f"The financial health index for {company.title()} in {year} is {fhi:.2f}."
    else:
        return f"No financial health index data available for {company.title()} in {year}."

def cash_flow_efficiency_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        efficiency = result["Cash Flow Efficiency"].values[0]
        return f"The cash flow efficiency for {company.title()} in {year} is {efficiency:.2f}."
    else:
        return f"No cash flow efficiency data available for {company.title()} in {year}."

def highest_market_share_query(year):
    result = data[data["Year"] == int(year)].sort_values("Market Revenue Share (%)", ascending=False)
    if not result.empty:
        top_company = result.iloc[0]["Company"]
        share = result.iloc[0]["Market Revenue Share (%)"]
        return f"The company with the highest market revenue share in {year} is {top_company} with {share:.2f}%."
    else:
        return f"No market revenue share data available for {year}."

def market_revenue_share_query(company, year):
    result = data[(data["Company"].str.lower() == company) & (data["Year"] == int(year))]
    if not result.empty:
        share = result["Market Revenue Share (%)"].values[0]
        return f"The market revenue share for {company.title()} in {year} is {share:.2f}%."
    else:
        return f"No market revenue share data available for {company.title()} in {year}."

def highest_revenue_growth_query(year):
    result = data[data["Year"] == int(year)].sort_values("Revenue Growth (%)", ascending=False)
    if not result.empty:
        top_company = result.iloc[0]
        company_name = top_company["Company"]
        growth = top_company["Revenue Growth (%)"]
        return f"The company with the highest revenue growth in {year} is {company_name} with a growth rate of {growth:.2f}%."
    else:
        return f"No revenue growth data available for the year {year}."


## Generate Responses

In [5]:
def generate_response(companies, metrics, years):
    if not companies or not metrics:
        return (
            "I couldn't understand your query. Please include a company and metric.\n"
            "Example: 'What is the revenue growth for Tesla in 2022?'"
        )

    responses = []
    for company in companies:
        for metric in metrics:
            if years:
                for year in years:
                    if metric == "revenue":
                        responses.append(total_revenue_query(company, year))
                    elif metric == "revenue growth":
                        responses.append(revenue_growth_query(company, year))
                    elif metric == "net income":
                        responses.append(net_income_query(company, year))
                    elif metric == "net income growth":
                        responses.append(net_income_growth_query(company, year))
                    elif metric == "profitability":
                        responses.append(profitability_query(company, year))
                    elif metric == "profitable":
                        responses.append(profitability_query(company, year))
                    elif metric == "financial health index":
                        responses.append(financial_health_index_query(company, year))
                    elif metric == "debt-to-asset ratio":
                        responses.append(debt_to_asset_ratio_query(company, year))
                    elif metric == "market revenue share":
                        responses.append(market_revenue_share_query(company, year))
                    else:
                        responses.append(f"Sorry, I couldn't fetch the {metric} for {company.title()} in {year}.")
            else:
                if metric == "market revenue share":
                    responses.append(f"Market revenue share requires a specific year for {company.title()}.")
                else:
                    responses.append(f"The metric '{metric}' requires a year. Please specify the year.")
    
    return "\n".join(responses)




In [6]:
query_functions = {
    "revenue": total_revenue_query,
    "revenue growth": revenue_growth_query,
    "net income": net_income_query,
    "net income growth": net_income_growth_query,
    "profitability": profitability_query,
    "profitable": profitability_query,
    "debt-to-asset ratio": debt_to_asset_ratio_query,
    "financial health index": financial_health_index_query,
    "highest revenue growth": highest_revenue_growth_query,
    "cash flow efficiency": cash_flow_efficiency_query,
    "market revenue share": market_revenue_share_query,
    
}


## Chatbot

In [7]:

def chatbot():
    print("Welcome to the Financial Insights Chatbot!")
    print("This chatbot provides financial insights for Microsoft, Tesla, and Apple.")
    print("You can ask questions about revenue, growth, profitability, financial health metrics, and more, covering the years 2021 to 2023.")
    print("\nExamples of valid queries:")
    print("1. 'What is the revenue growth for Tesla in 2022?'")
    print("2. 'What is the debt-to-asset ratio for Apple in 2022 and 2023?'")
    print("3. 'How profitable were Microsoft and Apple in 2022?'")
    print("\nLet me know how I can assist you!")


    while True:
        user_query = input("\nYour question (or type 'exit' to quit): ")
        if user_query.lower() == "exit":
            print("Thanks for talking to me. Goodbye!")
            break
        
        user_query = preprocess_query(user_query)
        company, metric, years = extract_information(user_query)
        
        if company and metric and years:
            response = generate_response(company, metric, years)
        else:
            response = (
                "I couldn't understand your query. "
                "Please specify the company, metric, and year(s). "
                "Example: 'What is the revenue growth for Tesla in 2022?'"
            )
        print(response)
        
chatbot()

Welcome to the Financial Insights Chatbot!
This chatbot provides financial insights for Microsoft, Tesla, and Apple.
You can ask questions about revenue, growth, profitability, financial health metrics, and more, covering the years 2021 to 2023.

Examples of valid queries:
1. 'What is the revenue growth for Tesla in 2022?'
2. 'What is the debt-to-asset ratio for Apple in 2022 and 2023?'
3. 'How profitable were Microsoft and Apple in 2022?'

Let me know how I can assist you!
Processed Query: revenue  tesla  2022  2023
Extracted Companies: ['tesla'], Metrics: ['revenue'], Years: ['2022', '2023']
The total revenue for Tesla in 2022 is $81462 million.
The total revenue for Tesla in 2023 is $96773 million.
Thanks for talking to me. Goodbye!
